<h1 align=center>Complete Image Classification Project: Tensorflow & PyTorch</h1>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

In [2]:
# =============================================================================
# TENSORFLOW IMPLEMENTATION
# =============================================================================
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

class TensorFlowImageClassifier:
    def __init__(self, num_classes=10):
        self.num_classes = num_classes
        self.model = None
        self.history = None
        self.class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                           'dog', 'frog', 'horse', 'ship', 'truck']

    def load_and_preprocess_data(self):
        """Load and preprocess CIFAR-10 data"""
        print("Loading CIFAR-10 dataset...")
        (x_train, y_train), (x_test, y_test) = cifar10.load_data()

        # Normalize pixel values
        x_train = x_train.astype('float32') / 255.0
        x_test = x_test.astype('float32') / 255.0

        # Convert labels to categorical
        y_train = to_categorical(y_train, self.num_classes)
        y_test = to_categorical(y_test, self.num_classes)

        print(f"Training data shape: {x_train.shape}")
        print(f"Testing data shape: {x_test.shape}")

        return x_train, y_train, x_test, y_test

    def build_model(self):
        """Build CNN model"""
        model = keras.Sequential([
            # First Conv Block
            layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
            layers.BatchNormalization(),
            layers.Conv2D(32, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),

            # Second Conv Block
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.Conv2D(64, (3, 3), activation='relu'),
            layers.MaxPooling2D((2, 2)),
            layers.Dropout(0.25),

            # Third Conv Block
            layers.Conv2D(128, (3, 3), activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.25),

            # Dense layers
            layers.Flatten(),
            layers.Dense(512, activation='relu'),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            layers.Dense(self.num_classes, activation='softmax')
        ])

        model.compile(
            optimizer='adam',
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )

        self.model = model
        return model

    def train(self, x_train, y_train, x_test, y_test, epochs=2, batch_size=32):
        """Train the model"""
        print("Training TensorFlow model...")

        # Callbacks
        callbacks = [
            keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
        ]

        self.history = self.model.fit(
            x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_test, y_test),
            callbacks=callbacks,
            verbose=1
        )

        return self.history

    def evaluate(self, x_test, y_test):
        """Evaluate model performance"""
        test_loss, test_acc = self.model.evaluate(x_test, y_test, verbose=0)
        print(f"TensorFlow Test Accuracy: {test_acc:.4f}")

        # Predictions
        y_pred = self.model.predict(x_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true_classes = np.argmax(y_test, axis=1)

        # Classification report
        print("\nClassification Report:")
        print(classification_report(y_true_classes, y_pred_classes,
                                  target_names=self.class_names))

        return test_acc, y_pred_classes, y_true_classes

In [3]:
# =============================================================================
# PYTORCH IMPLEMENTATION
# =============================================================================

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms

class PyTorchCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(PyTorchCNN, self).__init__()

        # First conv block
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d(0.25)

        # Second conv block
        self.conv3 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout2d(0.25)

        # Third conv block
        self.conv5 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.dropout3 = nn.Dropout2d(0.25)

        # Dense layers
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.bn4 = nn.BatchNorm1d(512)
        self.dropout4 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # First conv block
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = self.pool1(x)
        x = self.dropout1(x)

        # Second conv block
        x = F.relu(self.bn2(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = self.pool2(x)
        x = self.dropout2(x)

        # Third conv block
        x = F.relu(self.bn3(self.conv5(x)))
        x = self.dropout3(x)

        # Dense layers
        x = x.view(-1, 128 * 8 * 8)
        x = F.relu(self.bn4(self.fc1(x)))
        x = self.dropout4(x)
        x = self.fc2(x)

        return x

class PyTorchImageClassifier:
    def __init__(self, num_classes=10):
        self.num_classes = num_classes
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = PyTorchCNN(num_classes).to(self.device)
        self.class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                           'dog', 'frog', 'horse', 'ship', 'truck']
        print(f"Using device: {self.device}")

    def load_and_preprocess_data(self):
        """Load and preprocess CIFAR-10 data"""
        print("Loading CIFAR-10 dataset for PyTorch...")

        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        # Load CIFAR-10
        train_dataset = datasets.CIFAR10(root='./data', train=True,
                                       download=True, transform=transform)
        test_dataset = datasets.CIFAR10(root='./data', train=False,
                                      download=True, transform=transform)

        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

        return train_loader, test_loader

    def train(self, train_loader, test_loader, epochs=2):
        """Train the model"""
        print("Training PyTorch model...")

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.model.parameters())
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

        train_losses = []
        train_accuracies = []
        val_accuracies = []

        best_val_acc = 0
        patience_counter = 0

        for epoch in range(epochs):
            # Training
            self.model.train()
            running_loss = 0.0
            correct = 0
            total = 0

            for inputs, labels in train_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            train_acc = 100 * correct / total
            train_losses.append(running_loss / len(train_loader))
            train_accuracies.append(train_acc)

            # Validation
            val_acc = self.evaluate_epoch(test_loader)
            val_accuracies.append(val_acc)

            scheduler.step(running_loss / len(train_loader))

            print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}, '
                  f'Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')

            # Early stopping
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                patience_counter = 0
                torch.save(self.model.state_dict(), 'best_model.pth')
            else:
                patience_counter += 1
                if patience_counter >= 10:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

        # Load best model
        self.model.load_state_dict(torch.load('best_model.pth'))

        return train_losses, train_accuracies, val_accuracies

    def evaluate_epoch(self, test_loader):
        """Evaluate model for one epoch"""
        self.model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        return 100 * correct / total

    def evaluate(self, test_loader):
        """Evaluate model performance"""
        self.model.eval()
        all_predictions = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                outputs = self.model(inputs)
                _, predicted = torch.max(outputs, 1)

                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = 100 * sum(np.array(all_predictions) == np.array(all_labels)) / len(all_labels)
        print(f"PyTorch Test Accuracy: {accuracy:.4f}%")

        # Classification report
        print("\nClassification Report:")
        print(classification_report(all_labels, all_predictions,
                                  target_names=self.class_names))

        return accuracy, all_predictions, all_labels

In [4]:
# =============================================================================
# VISUALIZATION AND COMPARISON
# =============================================================================

def plot_training_history(tf_history, torch_losses, torch_train_acc, torch_val_acc):
    """Plot training history comparison"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))

    # TensorFlow plots
    axes[0, 0].plot(tf_history.history['loss'], label='Train Loss')
    axes[0, 0].plot(tf_history.history['val_loss'], label='Val Loss')
    axes[0, 0].set_title('TensorFlow - Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)

    axes[0, 1].plot(tf_history.history['accuracy'], label='Train Acc')
    axes[0, 1].plot(tf_history.history['val_accuracy'], label='Val Acc')
    axes[0, 1].set_title('TensorFlow - Accuracy')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True)

    # PyTorch plots
    axes[1, 0].plot(torch_losses, label='Train Loss')
    axes[1, 0].set_title('PyTorch - Loss')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Loss')
    axes[1, 0].legend()
    axes[1, 0].grid(True)

    axes[1, 1].plot(torch_train_acc, label='Train Acc')
    axes[1, 1].plot(torch_val_acc, label='Val Acc')
    axes[1, 1].set_title('PyTorch - Accuracy')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Accuracy')
    axes[1, 1].legend()
    axes[1, 1].grid(True)

    plt.tight_layout()
    plt.show()

def plot_confusion_matrices(y_true_tf, y_pred_tf, y_true_torch, y_pred_torch, class_names):
    """Plot confusion matrices for both models"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # TensorFlow confusion matrix
    cm_tf = confusion_matrix(y_true_tf, y_pred_tf)
    sns.heatmap(cm_tf, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names, ax=axes[0])
    axes[0].set_title('TensorFlow Confusion Matrix')
    axes[0].set_xlabel('Predicted')
    axes[0].set_ylabel('Actual')

    # PyTorch confusion matrix
    cm_torch = confusion_matrix(y_true_torch, y_pred_torch)
    sns.heatmap(cm_torch, annot=True, fmt='d', cmap='Reds',
                xticklabels=class_names, yticklabels=class_names, ax=axes[1])
    axes[1].set_title('PyTorch Confusion Matrix')
    axes[1].set_xlabel('Predicted')
    axes[1].set_ylabel('Actual')

    plt.tight_layout()
    plt.show()

def display_sample_predictions(x_test, y_true, y_pred_tf, y_pred_torch, class_names, n_samples=8):
    """Display sample predictions from both models"""
    fig, axes = plt.subplots(2, n_samples//2, figsize=(15, 6))
    axes = axes.flatten()

    for i in range(n_samples):
        # Denormalize image for display
        img = x_test[i]
        if img.max() <= 1:  # If normalized
            img = (img * 255).astype(np.uint8)

        axes[i].imshow(img)
        axes[i].set_title(f'True: {class_names[y_true[i]]}\n'
                         f'TF: {class_names[y_pred_tf[i]]}\n'
                         f'PT: {class_names[y_pred_torch[i]]}')
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

In [5]:
# =============================================================================
# MAIN EXECUTION
# =============================================================================

def main():
    """Main function to run the complete project"""
    print("=== Complete Image Classification Project ===")
    print("Comparing TensorFlow and PyTorch implementations\n")

    # Initialize classifiers
    tf_classifier = TensorFlowImageClassifier()
    torch_classifier = PyTorchImageClassifier()

    # Load data for TensorFlow
    x_train, y_train, x_test, y_test = tf_classifier.load_and_preprocess_data()

    # Build and train TensorFlow model
    tf_classifier.build_model()
    print(f"\nTensorFlow Model Summary:")
    tf_classifier.model.summary()

    tf_history = tf_classifier.train(x_train, y_train, x_test, y_test, epochs=20)
    tf_acc, tf_pred, tf_true = tf_classifier.evaluate(x_test, y_test)

    # Load data for PyTorch
    train_loader, test_loader = torch_classifier.load_and_preprocess_data()

    # Train PyTorch model
    torch_losses, torch_train_acc, torch_val_acc = torch_classifier.train(
        train_loader, test_loader, epochs=20)
    torch_acc, torch_pred, torch_true = torch_classifier.evaluate(test_loader)

    # Visualizations
    print("\n=== Generating Visualizations ===")

    # Plot training history
    plot_training_history(tf_history, torch_losses, torch_train_acc, torch_val_acc)

    # Plot confusion matrices
    plot_confusion_matrices(tf_true, tf_pred, torch_true, torch_pred,
                           tf_classifier.class_names)

    # Display sample predictions
    display_sample_predictions(x_test, tf_true, tf_pred, torch_pred,
                             tf_classifier.class_names)

    # Final comparison
    print(f"\n=== Final Results ===")
    print(f"TensorFlow Test Accuracy: {tf_acc:.4f}")
    print(f"PyTorch Test Accuracy: {torch_acc:.4f}")

    if tf_acc > torch_acc:
        print("TensorFlow model performed better!")
    elif torch_acc > tf_acc:
        print("PyTorch model performed better!")
    else:
        print("Both models performed equally well!")

if __name__ == "__main__":
    main()

=== Complete Image Classification Project ===
Comparing TensorFlow and PyTorch implementations

Using device: cpu
Loading CIFAR-10 dataset...
Training data shape: (50000, 32, 32, 3)
Testing data shape: (10000, 32, 32, 3)

TensorFlow Model Summary:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 30, 30, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 12, 12, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 10, 10, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 3, 3, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 737,834 (2.81 MB)

 Trainable params: 736,362 (2.81 MB)

 Non-trainable params: 1,472 (5.75 KB)

Training TensorFlow model...
Epoch 1/20
1441/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 124ms/step - accuracy: 0.3239 - loss: 2.1922

KeyboardInterrupt: 